In [26]:
import pyodbc 
import pandas as pd
from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)

In [24]:
# establish connection engine
server = 'DESKTOP-5CO5VP6'
database = 'liquorSales'
connection_string = fr"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

engine = create_engine(connection_string)

In [27]:
# read table as dataframe
query = "SELECT * FROM finance_liquor_sales"
df = pd.read_sql(query, engine)
df.head()

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-31797900035,2020-11-10,4967,Jeffs Market / Blue Grass,"102, W Mayne St",Blue Grass,52726.0,POINT (-90.766126 41.509119),82,SCOTT,1011600.0,Straight Rye Whiskies,255,Infinium Spirits,26858,Templeton 4YR Rye,12,375,9.99,14.99,1,14.99,0.37,0.09
1,INV-23548800092,2019-11-27,2601,Hy-Vee Food Store / Fairfield,1300 West Burlington Ave,Fairfield,52556.0,POINT (-91.978702 41.006456),51,JEFFERSON,1081500.0,Triple Sec,434,LUXCO INC,86251,Juarez Triple Sec,12,1000,2.42,3.63,6,21.78,6.00,1.58
2,INV-23609300026,2019-12-02,4962,Hilltop Grocery,1312 Harrison St,Davenport,52803.0,,82,SCOTT,1091200.0,Neutral Grain Spirits,434,LUXCO INC,41844,Everclear Alcohol,24,375,4.20,6.30,3,18.90,1.12,0.29
3,INV-39482900037,2021-08-24,3719,Wal-Mart 0581 / Marshalltown,2802 S Center St,Marshalltown,50158.0,POINT (-92.91231 42.012918),64,MARSHALL,1082100.0,Imported Cordials & Liqueur,192,"Mast-Jagermeister US, Inc",65259,Jagermeister Liqueur Mini Meisters,12,20,4.93,7.40,12,88.80,0.24,0.06
4,INV-39520400088,2021-08-25,5423,Stammer Liquor Corp,615 2nd Ave,Sheldon,51201.0,POINT (-95.854899 43.184242),71,OBRIEN,1092100.0,Imported Distilled Spirit Specialty,421,SAZERAC COMPANY INC,77487,Tortilla Gold DSS,12,1000,4.85,7.28,4,29.12,4.00,1.05


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   invoice_and_item_number  196 non-null    object        
 1   date                     196 non-null    datetime64[ns]
 2   store_number             196 non-null    int64         
 3   store_name               196 non-null    object        
 4   address                  196 non-null    object        
 5   city                     196 non-null    object        
 6   zip_code                 196 non-null    object        
 7   store_location           196 non-null    object        
 8   county_number            196 non-null    int64         
 9   county                   196 non-null    object        
 10  category                 196 non-null    float64       
 11  category_name            196 non-null    object        
 12  vendor_number            196 non-nul

In [46]:
#find the most popular item per zipcode
df_max = df.groupby('zip_code')['bottles_sold'].max()
pop_item = df.merge(df_max .rename('Max Bottles Sold'), left_on='zip_code', right_index=True)

# Filter the DataFrame to include only records with the same highest sold bottles
pop_item = pop_item[pop_item['bottles_sold'] == pop_item['Max Bottles Sold']]
pop_item

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons,Max Bottles Sold
0,INV-31797900035,2020-11-10,4967,Jeffs Market / Blue Grass,"102, W Mayne St",Blue Grass,52726.0,POINT (-90.766126 41.509119),82,SCOTT,1011600.0,Straight Rye Whiskies,255,Infinium Spirits,26858,Templeton 4YR Rye,12,375,9.99,14.99,1,14.99,0.37,0.09,1
1,INV-23548800092,2019-11-27,2601,Hy-Vee Food Store / Fairfield,1300 West Burlington Ave,Fairfield,52556.0,POINT (-91.978702 41.006456),51,JEFFERSON,1081500.0,Triple Sec,434,LUXCO INC,86251,Juarez Triple Sec,12,1000,2.42,3.63,6,21.78,6.00,1.58,6
2,INV-23609300026,2019-12-02,4962,Hilltop Grocery,1312 Harrison St,Davenport,52803.0,,82,SCOTT,1091200.0,Neutral Grain Spirits,434,LUXCO INC,41844,Everclear Alcohol,24,375,4.20,6.30,3,18.90,1.12,0.29,3
3,INV-39482900037,2021-08-24,3719,Wal-Mart 0581 / Marshalltown,2802 S Center St,Marshalltown,50158.0,POINT (-92.91231 42.012918),64,MARSHALL,1082100.0,Imported Cordials & Liqueur,192,"Mast-Jagermeister US, Inc",65259,Jagermeister Liqueur Mini Meisters,12,20,4.93,7.40,12,88.80,0.24,0.06,12
4,INV-39520400088,2021-08-25,5423,Stammer Liquor Corp,615 2nd Ave,Sheldon,51201.0,POINT (-95.854899 43.184242),71,OBRIEN,1092100.0,Imported Distilled Spirit Specialty,421,SAZERAC COMPANY INC,77487,Tortilla Gold DSS,12,1000,4.85,7.28,4,29.12,4.00,1.05,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,INV-44363100014,2022-02-02,5298,DeWitt Travel Mart,"630, 6th Ave",DeWitt,52742.0,POINT (-90.538952 41.821779),23,CLINTON,1091300.0,Neutral Grain Spirits Flavored,384,PIEDMONT DISTILLERS INC,76069,Midnight Moon Apple Pie Mini,4,50,17.16,25.74,4,102.96,0.20,0.05,4
185,S14178600031,2013-10-14,4098,Wal-Mart 1475 / Washington,2485 HWY 92,WASHINGTON,52353,POINT (-91.660194 41.296202),92,Washington,1701100.0,DECANTERS & SPECIALTY PACKAGES,421,"Sazerac Co., Inc.",76436,Old St. Nicks Egg Nog,12,750,3.64,5.46,120,655.20,90.00,23.78,120
186,S29111400017,2015-11-17,2565,Hy-Vee Food Store / Spencer,819 N GRAND AVE,SPENCER,51301,POINT (-95.14506 43.145897),21,Clay,1081330.0,PEACH SCHNAPPS,65,Jim Beam Brands,82847,Dekuyper Peachtree,12,1000,7.62,11.43,5,57.15,5.00,1.32,5
192,INV-33910800023,2021-01-29,2644,Hy-Vee Fort Dodge Wine and Spirits,1511 2nd Ave North,Fort Dodge,50501.0,POINT (-94.177165 42.508344),94,WEBSTER,1700000.0,Temporary & Specialty Packages,260,DIAGEO AMERICAS,101217,Captain Morgan OSR PET 6/1.75l w/ 50ml CM Sli...,6,1750,18.00,27.00,156,4212.00,273.00,72.11,156


In [47]:
# find the percentage of sales per store
total_sales_per_store = df.groupby('store_number')['sale_dollars'].sum()
total_overall_sales = df['sale_dollars'].sum()
percentage_sales_per_store = (total_sales_per_store / total_overall_sales) * 100

sales_per_store = pop_item.merge(percentage_sales_per_store.rename('Percentage Sales'), left_on='store_number', right_index=True)
sales_per_store['Percentage Sales'] = sales_per_store['Percentage Sales'].apply(lambda x: f'{x:.2f}%')
sales_per_store.head()

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons,Max Bottles Sold,Percentage Sales
0,INV-31797900035,2020-11-10,4967,Jeffs Market / Blue Grass,"102, W Mayne St",Blue Grass,52726.0,POINT (-90.766126 41.509119),82,SCOTT,1011600.0,Straight Rye Whiskies,255,Infinium Spirits,26858,Templeton 4YR Rye,12,375,9.99,14.99,1,14.99,0.37,0.09,1,0.01%
1,INV-23548800092,2019-11-27,2601,Hy-Vee Food Store / Fairfield,1300 West Burlington Ave,Fairfield,52556.0,POINT (-91.978702 41.006456),51,JEFFERSON,1081500.0,Triple Sec,434,LUXCO INC,86251,Juarez Triple Sec,12,1000,2.42,3.63,6,21.78,6.00,1.58,6,0.02%
2,INV-23609300026,2019-12-02,4962,Hilltop Grocery,1312 Harrison St,Davenport,52803.0,,82,SCOTT,1091200.0,Neutral Grain Spirits,434,LUXCO INC,41844,Everclear Alcohol,24,375,4.20,6.30,3,18.90,1.12,0.29,3,0.01%
3,INV-39482900037,2021-08-24,3719,Wal-Mart 0581 / Marshalltown,2802 S Center St,Marshalltown,50158.0,POINT (-92.91231 42.012918),64,MARSHALL,1082100.0,Imported Cordials & Liqueur,192,"Mast-Jagermeister US, Inc",65259,Jagermeister Liqueur Mini Meisters,12,20,4.93,7.40,12,88.80,0.24,0.06,12,0.06%
4,INV-39520400088,2021-08-25,5423,Stammer Liquor Corp,615 2nd Ave,Sheldon,51201.0,POINT (-95.854899 43.184242),71,OBRIEN,1092100.0,Imported Distilled Spirit Specialty,421,SAZERAC COMPANY INC,77487,Tortilla Gold DSS,12,1000,4.85,7.28,4,29.12,4.00,1.05,4,0.02%


In [48]:
# filter the year between 2016-2019
filtered_df = sales_per_store[(sales_per_store['date'].dt.year >= 2016) & (df['date'].dt.year <= 2019)]
filtered_df.head()

C:\Users\PC\AppData\Local\Temp\ipykernel_28568\1369448254.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = sales_per_store[(sales_per_store['date'].dt.year >= 2016) & (df['date'].dt.year <= 2019)]


,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons,Max Bottles Sold,Percentage Sales
1,INV-23548800092,2019-11-27,2601,Hy-Vee Food Store / Fairfield,1300 West Burlington Ave,Fairfield,52556.0,POINT (-91.978702 41.006456),51,JEFFERSON,1081500.0,Triple Sec,434,LUXCO INC,86251,Juarez Triple Sec,12,1000,2.42,3.63,6,21.78,6.00,1.58,6,0.02%
2,INV-23609300026,2019-12-02,4962,Hilltop Grocery,1312 Harrison St,Davenport,52803.0,,82,SCOTT,1091200.0,Neutral Grain Spirits,434,LUXCO INC,41844,Everclear Alcohol,24,375,4.20,6.30,3,18.90,1.12,0.29,3,0.01%
7,S30348700047,2016-01-25,3162,Nash Finch / Wholesale Food,807 GRANDVIEW,MUSCATINE,52761,POINT (-91.06411300000002 41.408437),70,Muscatine,1081330.0,PEACH SCHNAPPS,65,Jim Beam Brands,82847,Dekuyper Peachtree,12,1000,7.62,11.43,4,45.72,4.00,1.06,4,0.03%
8,S30466200002,2016-02-01,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14TH ST,DES MOINES,50320,POINT (-93.596754 41.554101),77,Polk,1081700.0,DISTILLED SPIRITS SPECIALTY,55,Sazerac North America,973627,Di Amore Quattro Orange,12,1000,9.75,14.63,120,1755.60,120.00,31.70,120,3.48%
60,S30624700060,2016-02-09,5166,East Side Liquor and Groceries,1116 E NEVADA ST,MARSHALLTOWN,50158,POINT (-92.893113 42.044345),64,Marshall,1052100.0,,420,"Moet Hennessy USA, Inc.",48099,Hennessy VS,24,200,5.74,8.61,24,206.64,4.80,1.27,24,0.15%


In [49]:
filtered_df.shape

(42, 26)

In [50]:
filtered_df.to_csv('liquor_sales_converted.csv', index=False)